# Dataset Building
Transmitting and saving the dataset.

This notebook:
Will read the dataset from `transmit_set/transmit_mods_SNR.pkl` and for each modulation scheme, loop over the `mods` and `snrs` to create a dataset where the samples have passed through the RFSoC in loopback with an Nooelec VeGA and lowpass filter (10-2500 MHz) in the loopback chain.

**Mods:**
`mods = ['QPSK', 'BPSK', 'QAM16', 'QAM64', 'PSK8', 'PAM4', 'GFSK', 'CPFSK']`

**SNRs:**
`snrs = [-20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20, 24, 28, 30]`

The resulting files will be saved as:
`received_{mod}.pkl`

In [ ]:
# import libraries
import numpy as np
import scipy.io as sci
import pickle
import plotly.graph_objects as go

In [ ]:
import xrfdc
import xrfclk
from pynq import allocate
from dataset_building.overlay import Overlay
ol = Overlay('bitstream/rfsoc_amc_qat.bit')

In [ ]:
mods = ['QPSK','BPSK','QAM16','QAM64','PSK8','PAM4','GFSK','CPFSK']
snrs = ['-20','-16','-12','-8','-4','0','4','8','12','16','20','24', '28', '30']

In [ ]:
# VeGA Switches set to D0,D2,D4 = 1
# Set phase offset array
phase_offset = range(-179, 179, 10)
mods = ['QPSK','BPSK','QAM16','QAM64','PSK8','PAM4','GFSK','CPFSK']
snrs = ['-20','-16','-12','-8','-4','0','4','8','12','16','20','24', '28', '30']

for mod in mods:
    print(f'Starting {mod}...')
    with open(f'./transmit_set/transmit_{mod.lower()}_SNR.pkl', 'rb') as f:
        dataset = pickle.load(f)
    data_dict = {}
    for snr in snrs:
        data_mod = dataset[mod,snr]
        # Loop through all the mods and transmit each
        complex_mod = np.array([])
        i_po = 0 # phase offset index
        for i in range(data_mod.shape[2]):
            data = data_mod[:,:,i]
            # multiply to fit number in int16 wordlength
            y = np.int16(data*np.int16(pow(2,13)))
            # Prepare sending variable
            z = np.zeros(2*4096, dtype=np.int16)
            # Interleave samples
            z[0::2] = y[0,:]
            z[1::2] = y[1,:]
            # Phase offset
            po = phase_offset[i_po%len(phase_offset)]
            ol.phase_offset_tx = po
            i_po += 1
            # Transmit data through DAC
            ol.send(z)
            
            # Receive multiple times.
            for j in range(16):
                # Receive data through ADC
                re_data, im_data = ol.receive_data()
                # Data received as s16_14
                complex_data = np.vstack([re_data, im_data])
                complex_split = complex_data
                # Stack onto accumulative variable to store in dict
                if complex_mod.size == 0:
                    complex_mod = complex_split
                else:
                    complex_mod = np.dstack([complex_mod, complex_split])
            # Stop sending data
            ol.stop()
            print('.',end='')
        # After looping through all frames - save to dict
        data_dict[mod, snr] = complex_mod
        
    # Write data to file
    with open(f'./received_set/loopback_train_{mod}.pkl', 'wb') as fpkl:
        pickle.dump(data_dict, fpkl, protocol=pickle.HIGHEST_PROTOCOL)
    print(' ')
    print(f'{mod} file written!')

In [ ]:
print('Dataset Saving...')
with open(f'./transmit_set/transmit_test_SNR.pkl', 'wb') as fpkl:
    pickle.dump(test_dict, fpkl, protocol=pickle.HIGHEST_PROTOCOL)
print('Saved!')

----